In [4]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2025-06-03 08:25:19--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2025-06-03 08:25:19 (74.2 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [14]:
import minsearch

In [15]:
import json

In [16]:
with open('documents.json', 'rt') as f_in: 
    docs_raw = json.load(f_in)

In [17]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [18]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [19]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [20]:
index.fit(documents)

In [21]:
q = 'the course has already started, can I still enroll?'
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict = boost,
    num_results = 5
)

In [35]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [71]:
from openai import OpenAI

In [72]:
import os

In [73]:
client = OpenAI()

In [74]:
q = "the course has already started, can I still enroll?"

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

In [75]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on the institution or platform offering the course. Here are a few steps you can take:\n\n1. **Check the Course Website**: Visit the course's official webpage for information on late enrollment policies.\n\n2. **Contact the Instructor or Administration**: Reach out directly to the course instructor or the administration office to inquire about the possibility of joining late.\n\n3. **Review the Course Syllabus**: Some courses may allow late enrollment if the initial weeks are introductory.\n\n4. **Consider Course Format**: Online courses often have more flexible enrollment policies than in-person classes.\n\n5. **Understand the Implications**: If late enrollment is an option, make sure you’re aware of any missed material or deadlines.\n\nTaking these steps should clarify whether late enrollment is possible for your specific situation."

In [76]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=q, context=context).strip()


In [77]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: Module 6: streaming with kafka
question: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
answer: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

section: Module 6: streaming with kafka
question: Module “kafka” not found when trying to run producer.py
answer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.
To create a virtual env and install packages (run only once)
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt
To activate it (you'll need to run it every time you need the virtual env):
sourc

In [78]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

'NONE'

In [110]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )

    return results

In [111]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [116]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [117]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [118]:
rag(query)

'To run Kafka, you can do the following depending on your scenario:\n\n1. **Java Kafka**: If you need to run a producer, consumer, or kstreams in the terminal using Java, navigate to the project directory and execute:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n2. **Python Kafka**: \n   - Make sure any necessary Docker images are up and running, and then create a virtual environment:\n     ```\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n   - Activate the environment whenever you need it with:\n     ```\n     source env/bin/activate\n     ```\n   - And deactivate it when done:\n     ```\n     deactivate\n     ```\n   - For scripts like `producer.py`, this setup helps run Python files related to Kafka.\n\n3. **Permission Issue**: If you encounter a `Permission denied` error when using `./build.sh`, resolve it by running:\n   ```\n   chmod +x build.sh\n

In [119]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even if it has already started. You are eligible to submit the homeworks. However, be mindful that there will be deadlines for turning in the final projects, so it's best not to leave everything until the last minute."